<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Определение формы иона молекулы водорода
## Examples - Quantum Mechanics
<section class="post-meta">
By Henning G. Hugdal, Magnus H-S Dahle, Håkon W. Ånes and Peter Berg
</section>
Last edited: January 19th 2019
___

Цель этого примера состоит в том, чтобы определить форму иона молекулы водорода. Задача состоит в том, чтобы использовать интегрирование по методу Монте-Карло для нахождения соотношения между полуосями эллипсоида таким образом, чтобы вероятность нахождения электрона внутри эллипсоида была максимальной. Если вы не знакомы с интегрированием Монте-Карло, взгляните на краткое введение, приведенное в этом [блокноте](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/ex_qm3_the_size_of_a_hydrogen_atom_monte_carlo_integration.ipynb).

Нас интересует определение формы иона молекулы водорода, то есть одного электрона, связанного с двумя протонами. Гамильтониан для этой системы равен

$$H = -\frac{\hbar^2}{2m}\nabla^2 - \frac{e^2}{4 \pi \epsilon_o}\left(\frac{1}{r_1}+\frac{1}{r_2}\right). $$
    
Используя волновую функцию 1s $\psi_{100}$ для водорода в качестве основы, мы предлагаем пробную функцию

$$ \psi = A[\psi_{100}(r_1)+\psi_{100}(r_2)].$$

Константа нормализации равна (см., например, Гриффитс, стр. 306 [[1]](#rsc))
$$ A = \sqrt{\frac{1}{2(1+I)}}$$
где 
$$ I = \exp(-R/a)\left[1+\frac{R}{a}+\frac{1}{3}\left(\frac{R}{a}\right)^2\right],$$

где $R$ - расстояние между протонами. Как рассчитано на странице 308 в Griffiths [[1]](#rsc), расстояние, минимизирующее энергию, составляет $R=2.4 a$, где $a= 0.529$ Å-радиус Бора. Однако, чтобы избежать ошибок округления, мы установим $a=1$.

Мы предполагаем азимутальную симметрию, т.е. отсутствие $\phi$-зависимости, такой, что полуосновные оси в двух направлениях, скажем, в направлениях $x$ и $y$, равны. Затем два протона помещаются на ось $z$, например, в точке $\pm R/2 \hat{z}$. Для заданного объема эллипсоида
$$V_0 = \frac{4}{3}\pi(2a)^3,$$
мы хотим найти соотношение между полуосновными осями $b$ и $c$ эллипсоида, которое максимизирует вероятность нахождения электрона внутри эллипсоида.

Нам также понадобится, чтобы поверхность эллипсоида описывалась
$$ \frac{x^2}{b^2}+\frac{y^2}{b^2}+\frac{z^2}{c^2} = 1$$
и что объем задается
$$ V = \frac{4}{3}\pi b^2 c.$$

Теперь мы можем начинать!

In [ ]:
%matplotlib inline
#from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import random
import matplotlib.collections as collections

# Set common figure parameters:
newparams = {'axes.labelsize': 14, 'axes.linewidth': 1, 'savefig.dpi': 200, 
             'lines.linewidth': 1.0, 'figure.figsize': (2, 2),
             'ytick.labelsize': 10, 'xtick.labelsize': 10,
             'ytick.major.pad': 5, 'xtick.major.pad': 5,
             'legend.fontsize': 10, 'legend.frameon': False, 
             'legend.handlelength': 1.5, 'figure.dpi': 150}
plt.rcParams.update(newparams)

Прежде всего, мы определяем функцию `psi(r1,r2, R)`, которая возвращает значение волновой функции для заданных расстояний $r_1$ и $r_2$ до двух протонов и расстояние $R$ между протонами.

In [ ]:
def psi(r1, r2, R): 
    """Wavefunction for an electron in a potential from two protons.
    Input:
        r1     Distance to proton 1
        r2     Distance to proton 2
        R      Distance between protons"""
    a = 1 # Bohr radius (set to one)
    I = np.exp(-R/a)*(1+R/a+1/3*(R/a)**2)
    A = np.sqrt(1/(2*(1+I)))
    return A/np.sqrt(np.pi*a**3)*(np.exp(-r1/a)+np.exp(-r2/a))

Затем мы определяем количество случайных точек, $N$, и используем общую процедуру, описанную [здесь](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/ex_qm3_the_size_of_a_hydrogen_atom_monte_carlo_integration.ipynb) для вычисления вероятности внутри эллипсоида с полуосновными осями $b$ и $c$. Поскольку объем эллипсоида должен быть равен объему $V_0$, $c$ определяется из выбранного значения $b$.

In [ ]:
N = 1.0e5 # Number of random numbers
a = 1     # Bohr radius (set to one)
R = 2.4*a # From Griffiths p. 308
i = 0
n = 0
V_0 = (4/3)*np.pi*(2*a)**3  # Given volume

В следующем коде мы находим значение $b$, которое максимизирует вероятность. $2a$ выбрано в качестве верхней границы, поскольку представляется разумным, что эллипсоид должен быть растянут в направлении $z$, что означает $b<c$ и, следовательно, $b<2a$, что соответствует сфере с заданным объемом $V_0$.

In [ ]:
"""The following code calculates the value of b which maximizes the probability inside the ellipsoid.
This is however not a very efficient solution."""

b_min = 1*a  # Lower limit for b
b_max = 2*a  # Upper limit for b, corresponds to sphere
b_steps = 6  # Number of steps between b_min and b_max
b_ = np.linspace(b_min, b_max, b_steps)  # Array of b-values, the lenght of semi-principal axis 
                                         # in the x- and y-direction.
acc = 0.0001*a  # Wanted accuracy
prob = np.zeros(b_steps)
while (b_max-b_min) > acc:
    for j, b in enumerate(b_):
        c = V_0*3/(4*np.pi*b**2)   # Length of semi-principal axis in z-direction, calculated from V_0 and b
        while i < N:
            x = random.uniform(-b, b)
            y = random.uniform(-b, b)
            z = random.uniform(-c, c)
            check = (x/b)**2+(y/b)**2+(z/c)**2 # Used to check if point is inside ellipsoid
            r1 = np.sqrt(x**2 + y**2 + (z-R/2)**2)
            r2 = np.sqrt(x**2 + y**2 + (z+R/2)**2)
            if check<=1:
                n = n + abs(psi(r1, r2, R))**2
            i = i + 1
        prob[j] = n/N*8*b**2*c
        n = 0
        i = 0
    b_max = b_[max(prob)==prob][0] + (b_max-b_min)/b_steps
    b_min = b_[max(prob)==prob][0] - (b_max-b_min)/b_steps
    b_ = np.linspace(b_min, b_max, b_steps)
    
prob_max = max(prob)
b = b_[prob_max==prob][0]  
c = V_0*3/(4*np.pi*b**2)
print("Maximum probability is: %s" % prob_max)
print("b/a = %s" % (b/a))
print("c/a = %s" % (c/a))
print("The ratio b/c is: %s" % (b/c))

Значение отношения $b/c$, которое дает наибольшую вероятность, составляет приблизительно 0.6, что означает, что эллипсоид довольно сильно растянут вдоль оси $z$. В качестве иллюстрации того, кажется ли это разумным, ниже показана плотность вероятности в плоскости $xz$ вместе с эллипсом с соотношением $c/b$, найденным выше. 

In [ ]:
p = 1000
xs = np.linspace(-1.1*c, 1.1*c, p, True)
X,Z = np.meshgrid(xs, xs)
psi2 = np.zeros([p, p])

r1 = np.sqrt(X**2+(Z-R/2)**2)
r2 = np.sqrt(X**2+(Z+R/2)**2)

psi2 = abs(psi(r1, r2, R))**2

plt.figure(figsize=(6,4.5))
levels = np.linspace(0, 1, 100, True)
C = plt.contourf(X/a, Z/a, psi2/psi(0,R,R)**2, levels)
plt.title('Probability density for hydrogen molecule ion')
plt.ylabel(r'$z/a$')
plt.xlabel(r'$x/a$')
cbar = plt.colorbar(C)
cbar.ax.set_ylabel('Probability density (relative at maximum)')


x = lambda v: b/a*np.cos(v)
z = lambda v: c/a*np.sin(v)
theta = np.linspace(0, 2*np.pi, 1000)
p1, = plt.plot(x(theta), z(theta), 'r')
p2, = plt.plot([0,0], [-R/a/2,R/a/2], 'r+')

Как мы видим, определенное соотношение кажется вполне разумным! Чтобы проверить полученный результат, также интересно численно интегрировать плотность вероятности, используя встроенную функцию из `scipy.integrate`. Это делается ниже с помощью функций `dblquad` и `tplquad`, которые позволяют интегрировать в двух и трех измерениях соответственно. Двумерная интегральная функция может быть использована, поскольку у нас есть азимутальная симметрия, что означает, что интегрирование по $\phi$ вносит только фактор $2\pi$. Это также, безусловно, самый эффективный код для запуска.

In [ ]:
from scipy.integrate import dblquad # Two dimensional integral function

def f2D(r, theta):
    a = 1.0 # Bohr radius (set to one)
    I = np.exp(-R/a)*(1+R/a+1/3*(R/a)**2)
    A = np.sqrt(1/(2*(1+I)))
    f = (A/np.sqrt(np.pi*a**3)*(np.exp(-np.sqrt(r**2+R**2/4-R*r*np.cos(theta))/a)\
        +np.exp(-np.sqrt(r**2+R**2/4+R*r*np.cos(theta))/a)))**2*r**2*np.sin(theta)
    return f

#Integration limits
r1 = 0
r2 = lambda theta: b*c/np.sqrt(c**2*np.sin(theta)**2+b**2*np.cos(theta)**2)

t1 = 0
t2 = np.pi

I = 2*np.pi*dblquad(f2D, t1, t2, lambda theta: r1, lambda theta: r2(theta))[0]
print("The probability is: %s" % I)

In [ ]:
from scipy.integrate import tplquad  # Three dimensional integral function

def f3D(phi, r, theta):
    a = 1.0 # Bohr radius (set to one)
    I = np.exp(-R/a)*(1+R/a+1/3*(R/a)**2)
    A = np.sqrt(1/(2*(1+I)))
    f = (A/np.sqrt(np.pi*a**3)*(np.exp(-np.sqrt(r**2+R**2/4-R*r*np.cos(theta))/a)+\
        np.exp(-np.sqrt(r**2+R**2/4+R*r*np.cos(theta))/a)))**2*r**2*np.sin(theta)
    return f

#Integration limits
r1 = 0
r2 = lambda theta: b*c/np.sqrt(c**2*np.sin(theta)**2+b**2*np.cos(theta)**2)

t1 = 0
t2 = np.pi

p1 = 0
p2 = 2*np.pi

I = tplquad(f3D,t1,t2,lambda theta: r1, lambda theta: r2(theta),lambda theta,r: p1, lambda theta,r: p2)[0]
print("The probability is: %s" % I)

Мы видим, что результат, полученный с помощью интеграции Монте-Карло, довольно хорошо соответствует результатам, полученным с помощью встроенных функций.

В качестве проверки наших предыдущих результатов мы также можем попытаться найти оптимальное решение, используя функцию `optimize.minimize` из библиотеки `scipy`. Здесь мы используем метод двумерной интеграции, поскольку он был наиболее эффективным.

In [ ]:
from scipy.optimize import minimize

def obj_func(b, V):
    """Objective function which returns the probability for a given value of the semi-principal axis b"""
    B = b*a
    C = V*3/(4*np.pi*B**2)
    r1 = 0
    r2 = lambda theta: B*C/np.sqrt(C**2*np.sin(theta)**2+B**2*np.cos(theta)**2)

    t1 = 0
    t2 = np.pi

    I = 2*np.pi*dblquad(f2D, t1, t2, lambda theta: r1, lambda theta: r2(theta))[0]
    return -I

def optimizeRatio(V, tol):
    b_0 = 0.01
    res = minimize(fun=obj_func, x0=b_0, args=(V,), jac=False, tol=tol)
    ratio = res.x[0]*a/(V*3/(4*np.pi*res.x[0]**2)/a**2)
    
    return (ratio, res)

rat, res = optimizeRatio(V_0, 1e-5)

print(res.message)
print("b/a = %s" % res.x[0])
print("Maximum probability = %s" % res.fun*(-1))
print("Ratio b/c = %s" % rat)

Как мы видим, результаты довольно хорошо согласуются с тем, что мы обнаружили ранее.
___

Следующий интересный вопрос: как изменяется соотношение $b/c$ при изменении объема $V_0$ эллипсоида? Размышляя о том, как расположены протоны, можно предположить, что $b/c$ будет уменьшаться при уменьшении объема, т.е. эллипсоид становится очень узким, чтобы вместить два протона. Для больших объемов $b/c$ должен приближаться к единице. Давайте проверим эти предположения. Чтобы сэкономить вычислительное время, мы будем использовать функцию `minimize` из библиотеки `scipy`, как было сделано выше.

In [ ]:
V = np.logspace(-3, 3, 7)
ratios = np.zeros(np.size(V))
results = np.zeros(np.size(V))
# We need higher accuracy when the volume gets larger, in order to find the correct ratio:

for i, V_ in enumerate(V):
    (ratios[i], res) = optimizeRatio(V_, 1e-4)

In [ ]:
plt.figure(figsize=(8,3))
plt.loglog(V, ratios, '-')
plt.ylabel(r"$b/c$")
plt.xlabel(r"$V$");

Мы видим, что наша догадка, похоже, вполне соответствует действительности!

___
<a id="rsc"></a>
## References

<a>[1]</a> Griffiths, D. J. _Introduction to Quantum Mechanics_. Pearson Education, 2004.